In [1]:
!pip install tensorflow iree-base-compiler iree-base-runtime iree-tools-tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 MB 702.1 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 608.8 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip freeze | grep -E 'iree|tensorflow|tf'

iree-base-compiler==3.1.0
iree-base-runtime==3.1.0
iree-tools-tf @ file:///mnt/home/douliyang/mlir-workspace/iree/integrations/tensorflow/python_projects/iree_tf
iree-tools-tflite @ file:///mnt/home/douliyang/mlir-workspace/iree/integrations/tensorflow/python_projects/iree_tflite
platformdirs==4.3.6
tensorflow==2.18.0
tensorflow-io-gcs-filesystem==0.37.1
tensorflow_cpu==2.18.0


In [12]:
!mkdir models3
!cd models3 && curl -L -o mobilenetv3.tar.gz https://www.kaggle.com/models/google/mobilenet-v3/TensorFlow2/small-075-224-classification/1/download && tar xf mobilenetv3.tar.gz
!ls -lah models3/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 7551k  100 7551k    0     0   146k      0  0:00:51  0:00:51 --:--:-- 94154
total 8.5M
drwxrwxr-x 3 douliyang douliyang 4.0K Feb  5 22:33 .
drwxrwxr-x 4 douliyang douliyang 4.0K Feb  5 22:32 ..
-rw-rw-r-- 1 douliyang douliyang 7.4M Feb  5 22:33 mobilenetv3.tar.gz
-rwx------ 1 douliyang douliyang 1.1M Feb 24  2023 saved_model.pb
drwxr-x--x 2 douliyang douliyang 4.0K Feb 24  2023 variables


In [13]:
import tensorflow.compat.v2 as tf
model = tf.saved_model.load('./models3/')
print(list(model.signatures.keys()))

['serving_default']


In [14]:
!iree-import-tf \
  --tf-import-type=savedmodel_v1 \
  --tf-savedmodel-exported-names=serving_default \
  ./models3/ -o iree_input.mlir

2025-02-05 22:34:35.421339: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-05 22:34:35.472991: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-05 22:34:38.008643: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./models3/
2025-02-05 22:34:38.017840: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-05 22:34:38.017868: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./models3/
I0000 00:00:1738766078.08958

In [15]:
!which iree-ir-tool
!iree-ir-tool cp iree_input.mlir -o iree_input_text.mlir

/home/douliyang/large/mlir-workspace/iree/iree/bin/iree-ir-tool


In [18]:
!tail -n 20 iree_input_text.mlir

    %594 = stablehlo.dynamic_reshape %593, %from_elements_30 : (tensor<?xf32>, tensor<2xindex>) -> tensor<?x1xf32>
    %595 = shape.shape_of %592 : tensor<?x1001xf32> -> tensor<2xindex>
    %596 = shape.cstr_broadcastable %595, %from_elements_30 : tensor<2xindex>, tensor<2xindex>
    %597 = shape.assuming %596 -> (tensor<?x1001xf32>) {
      %602 = shape.shape_of %592 : tensor<?x1001xf32> -> tensor<2xindex>
      %603 = shape.broadcast %602, %from_elements_30 : tensor<2xindex>, tensor<2xindex> -> tensor<2xindex>
      %604 = stablehlo.dynamic_broadcast_in_dim %592, %603, dims = [0, 1] : (tensor<?x1001xf32>, tensor<2xindex>) -> tensor<?x1001xf32>
      %605 = stablehlo.dynamic_broadcast_in_dim %594, %603, dims = [0, 1] : (tensor<?x1xf32>, tensor<2xindex>) -> tensor<?x1001xf32>
      %606 = stablehlo.subtract %604, %605 : tensor<?x1001xf32>
      shape.assuming_yield %606 : tensor<?x1001xf32>
    }
    %598 = stablehlo.exponential %597 : tensor<?x1001xf32>
    %599 = stablehlo.reduce(%59

In [19]:
!iree-compile --iree-hal-target-backends=cuda --iree-cuda-target=sm_86 iree_input.mlir -o mobilenet_spirv.vmfb

-:269:14: error: failed to legalize operation 'stablehlo.dynamic_broadcast_in_dim' that was explicitly marked illegal
-:269:14: note: see current operation: %2591 = "stablehlo.dynamic_broadcast_in_dim"(%176, %2590) <{broadcast_dimensions = array<i64: 0, 1, 2, 3>}> : (tensor<?x112x112x16xf32>, tensor<4xindex>) -> tensor<?x112x112x16xf32>
